# Aula 4 - Machine Learning

# Demo 9 - Algoritmo Apriori - Minerando a Escolha de Filmes

Como visto em aula, o algoritmo Apriori é baseado na noção de Regras de Associação. Tais regras são usadas, principalmente, para a correta identificação de relações subjacentes entre diferentes itens. Nesta demo trabalhamos com um exemplo de uma plataforma de filmes, na qual os clientes têm por opção o aluguel e (ou) compra de filmes.

Assim temos então uma premissa: geralmente vai existir um padrão no que os clientes compram; por exemplo, o tema Super-herói junto a filmes da categoria Crianças. Pensem então que, num caso de uso real, maior lucro poderia ser gerado a medida que forem melhor identificadas as relações entre os diferentes filmes. Ou seja, se os filmes A e B são frequentemente comprados juntos, esse padrão pode ser explorado para se aumentar o lucro.

Assim, a partir do padrão encontrado, as pessoas que compram ou alugam um desses dois filmes em particular podem ser levadas a alugar ou comprar o outro, por meio de campanhas ou sugestões na plataforma. Hoje epecialmente, estamos cada vez mais familiarizados com esses mecanismos de recomendação, principalmente na Netflix e Amazon, para citar os mais importantes.



## Teoria do Algoritmo Apriori

Existem três componentes principais do algoritmo Apriori:

- Suporte (*support*)
- Confiança (*confidence*)
- Lift

Vamos analisar cada componente. Antes de começarmos, precisamos verificar qual a janela de tempo que faz sentido de acordo com nosso regra de negócios. Então, em nosso exemplo, vão ser usados todos os filmes comprados ou alugados por clientes individuais em um mês ou um ano.

### Support

O suporte, para nosso caso de uso, refere-se então à popularidade de um filme e é calculado de acordo com o número de vezes que um determinado filme é assistido, dividido pelo número total de transações.

Por exemplo, se em 100 transações, 25 transações contiverem o filme "Blade Runner", o suporte para pode ser calculado como:

Suporte ("Blade Runner") = (Transações que contêm "Blade Runner") / (Total de transações)

Suporte ("Blade Runner") = 25/100 = 25%

### Confidence

Confiança refere-se à probabilidade de o filme "Blade Runner" ser comprado ou alugado se o filme "Eu Robô" for comprado ou alugado. Pode ser calculado ao se encontrar o número de transações em que "Blade Runner" e "Eu Robô" foram comprados juntos, dividido pelo número total de transações em que "Eu Robô" é comprado ou alugado.

Confiança ("Eu Robô" → "Blade Runner") = (Transações contendo ambos ("Eu Robô" e "Blade Runner")) / (Transações contendo "Eu Robô")

Se tivéssemos 10 transações em que os clientes assistiram "Eu Robô" e "Blade Runner", enquanto em 20 transações, "Eu Robô" é comprado ou alugado. Então podemos encontrar as chances de que "Blade Runner" seja comprado/alugado quando  "Eu Robô" é comprado.

Confiança ("Eu Robô" → "Blade Runner") = 20/20
                             = 50%

#### Lift

Em nosso caso, quando falamos de lift, estamos falando do aumento na venda de um determinado item quando algum outro também é vendido junto a este.

Por exemplo, o Lift ("Eu Robô" -> "Blade Runner") refere-se ao aumento na proporção da venda/aluguel do filme "Blade Runner" quando "Eu Robô" é vendido. Assim, o Lift pode ser calculado dividindo-se Confiança ("Eu Robô" -> "Blade Runner") dividido pelo Suporte ("Blade Runner").

Matematicamente, temos a seguinte representação:

Lift ("Eu Robô" → "Blade Runner") = (Confiança ("Eu Robô" → "Blade Runner")) / (Suporte ("Blade Runner"))
Pode ser calculado como:

Podemos ter, por exemplo:

Lift ("Eu Robô" → "Blade Runner") = 50% / 25%
                      = 2

Então, basicamente, o Lift nos diz que a probabilidade de compra/aluguel dos filmes "Eu Robô" e "Blade Runer" juntos é 2 vezes maior do que a probabilidade de compra/aluguel apenas do filme "Blade Runner".

Lembrete: Um aumento de 1 significa que não há associação entre produtos. Um valor de Lift superior a 1 significa que os produtos têm maior probabilidade de serem comprados juntos. Por fim, Lift inferior a 1 refere-se ao caso em que é improvável que dois produtos sejam comprados juntos.

### Etapas no Algoritmo Apriori

Para grandes conjuntos de dados, podem haver centenas de itens em centenas de milhares de transações. O algoritmo Apriori tenta extrair regras para cada combinação possível de itens. Por exemplo, o aumento pode ser calculado para o item 1 e item 2, item 1 e item 3, item 1 e item 4 e, em seguida, item 2 e item 3, item 2 e item 4 e, em seguida, combinações de itens, por exemplo. item 1, item 2 e item 3; Da mesma forma, item 1, item2 e item 4, e assim por diante.

Como você já deve ter pensado, esse processo pode ser extremamente lento, principalmente dado ao número de combinações.

Para que esse processo seja mais rápido, precisamos executar as seguintes etapas:

1. Define-se um valor mínimo para suporte e confiança.

Isso significa que estamos interessados apenas em encontrar regras para os itens que possuem certa existência padrão (por exemplo, suporte) e têm um valor mínimo para co-ocorrência com outros itens (confiança).

2. Extraiem-se todos os subconjuntos com maior valor de suporte que o limite mínimo.

3. Selecionam-se então todas as regras dos subconjuntos com valor de confiança maior que o limite mínimo.

4. Ordenam-se as regras por ordem decrescente de valor de Lift.



### Implementação de nosso Algoritmo Apriori

Agora, como visto em sala, usaremos o algoritmo Apriori para encontrar regras que descrevam associações entre diferentes produtos, dadas 7500 transações ao longo de um mês. O conjunto de dados dos filmes é escolhido aleatoriamente, não sendo dados reais.

Tendo em vista o cálculo de suporte, confiança e lift para todas as combinações possíveis de itens usaremos uma biblioteca chamada apyori, a qual é instalada em nosso ambiente do Google Colab por meio do seguinte trecho de código:

In [1]:
!pip3 install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=e505396c5a455f999889588eacb6fa98cfe4d6bfd02e2944fd137fc374155380
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


#### Importando nossas bibliotecas

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori

#### Importando nosso dataset

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():


  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving movie_dataset.csv to movie_dataset.csv
User uploaded file "movie_dataset.csv" with length 462989 bytes


In [4]:
movie_data = pd.read_csv('movie_dataset.csv', header = None)
num_records = len(movie_data)
print(num_records)

7501


Agora, usaremos o algoritmo Apriori para descobrir quais itens são comumente vendidos juntos, para que o proprietário da loja possa tomar medidas para colocar os itens relacionados juntos ou fazer publicidade de forma conjunto, tendo em vista aumentar o lucro.

#### Pré-processamento de Dados

A biblioteca Apriori que vamos usar exige que nosso conjunto de dados esteja na forma de uma lista de listas, onde todo o conjunto de dados é uma grande lista, e cada transação no conjunto de dados é uma lista interna da grande lista externa.

Atualmente, temos dados na forma de um dataframe pandas. Para converter nosso dataframe do pandas em uma lista de listas, execute o seguinte:

In [5]:
records = []
for i in range(0, num_records):
    records.append([str(movie_data.values[i,j]) for j in range(0, 20)])

#### Aplicando o Apriori

Agora podemos especificar os parâmetros da classe apriori.

- A lista
- min_support
- min_confidence
- min_lift
- min_length (o número mínimo de itens que você deseja em suas regras, geralmente 2)

Vamos supor que queremos apenas filmes comprados pelo menos 30 vezes em um mês. O suporte para esses itens pode ser calculado como 30/7500 = 0,004. A confiança mínima para as regras é de 20% ou 0,2. Da mesma forma, especificamos o valor de lift como 3 e, finalmente, min_length é 2, pois queremos pelo menos dois produtos em nossas regras.

Lembrete: Como falado em aula, esses valores geralmente são escolhidos de forma arbitrária e precisam ser ajustados empiricamente.

Agora, execute a seguinte célula

In [6]:
association_rules = apriori(records, min_support=0.004, min_confidence=0.20, min_lift=3, min_length=2)
association_results = list(association_rules)

Na segunda linha aqui, podem observar que convertemos as regras encontradas pela classe apriori em uma lista, pois é mais fácil visualizar os resultados neste formato.

#### Visualizando os resultados:

Agora, vamos primeiro encontrar o número total de regras extraídas pela classe apriori, de acordo com o seguinte script:

In [7]:
print(len(association_results))

58


O script acima deve retornar 56. Cada item correspondendo a uma regra.

Vamos imprimir o primeiro item na lista association_rules para ver a primeira regra gerada:

In [8]:
print(association_results[0])

RelationRecord(items=frozenset({'London Has Fallen', 'Captain America'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'London Has Fallen'}), items_add=frozenset({'Captain America'}), confidence=0.29059829059829057, lift=4.84395061728395)])


O primeiro item da lista é uma lista em si que contém três itens. O primeiro item da lista mostra os filmes na regra.

Por exemplo, desde o primeiro item, podemos ver que "London Has Fallen"" e "Captain America" são comumente comprados juntos.

O valor do suporte para a primeira regra é 0,0045. Esse número é calculado dividindo-se o número de transações que contêm "London Has Fallen", dividido pelo número total de transações.

O nível de confiança para a regra é 0,2905 o que mostra que, de todas as transações que contêm "London Has Fallen", quase 30% das transações também contêm "Captain America".

Por fim, o aumento de 4,84 nos diz que o "Captain America" é 4,84 vezes mais provável de ser comprado pelos clientes que compram "London Has Fallen" em comparação com a probabilidade padrão de venda do "Captain America".

O script a seguir exibe a regra em um dataframe, de maneira mais legível:



In [9]:
results = []
for item in association_results:

    pair = item[0] #primeiro indice da lista interna
    items = [x for x in pair] #contem item base e adiciona item

    value0 = str(items[0])
    value1 = str(items[1])

    #indice da lista interna
    value2 = str(item[1])[:7]

    #primeiro indice da lista localizada na posicao 0
    #do tercerio indice da lista interna

    value3 = str(item[2][0][2])[:7]
    value4 = str(item[2][0][3])[:7]

    rows = (value0, value1,value2,value3,value4)
    results.append(rows)

labels = ['Titulo 1','Titulo 2','Support','Confidence','Lift']
movie_suggestion = pd.DataFrame.from_records(results, columns = labels)

print(movie_suggestion)

             Titulo 1               Titulo 2  Support Confidence     Lift
0   London Has Fallen        Captain America  0.00453    0.29059  4.84395
1         Red Sparrow          Green Lantern  0.00573    0.30069  3.79083
2           Star Wars          Green Lantern  0.00586    0.37288  4.70081
3       Kung Fu Panda                Jumanji  0.01599    0.32345  3.29199
4        Wonder Woman                Jumanji  0.00533    0.37735  3.84065
5         Spiderman 3  The Spy Who Dumped Me  0.00799    0.27149  4.12241
6        The Revenant              Star Wars  0.00506    0.32203  4.50667
7                 nan      London Has Fallen  0.00453    0.29059  4.84395
8        The Revenant                 Intern  0.00533    0.23255  3.25451
9          Tomb Rider        Fantastic Beast  0.00479    0.57142  3.28199
10      Kung Fu Panda                Get Out  0.00413    0.20666  4.17845
11                nan            Red Sparrow  0.00573    0.30069  3.79083
12                nan              Sta